In [1]:
# read in some helpful libraries
import nltk                       # the natural langauage toolkit, open-source NLP
import pandas as pd               # pandas dataframe
import re                         # regular expression
from nltk.corpus import stopwords  
from gensim import parsing        # Help in preprocessing the data, very efficiently
import gensim
import numpy as np
import itertools
import os
from nltk.corpus import stopwords  
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.preprocessing.text import Tokenizer
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
print('import done')

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

C:\Users\khmar\ana\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


import done
You have TensorFlow version 1.13.1


In [2]:
DATA_FILE = 'C:/Users/khmar/Desktop/ISSUE/dataset/CSV/data_ameliorate/data.csv'
df = pd.read_csv(DATA_FILE,delimiter=';',encoding='UTF-8')
print(df.head())

                                                text     label
0  I had ordered a data cable, got a very well fi...  NOTISSUE
1                                   Love This Phone.  NOTISSUE
2                I get a very well finished product.  NOTISSUE
3                            I could not be happier.  NOTISSUE
4  I was looking for this headset for a long time...  NOTISSUE


In [3]:
texts= df['text']
tags = df['label']

In [4]:
# Confirm that we have a balanced dataset
# Note: data was randomly shuffled in our BigQuery query
df['label'].value_counts()

NOTISSUE    2030
ISSUE       2025
Name: label, dtype: int64

In [5]:
num_max = 1000
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')


In [7]:
#tok save
import pickle
tok_file = 'CNN_token_normal_embeddings_without_text_preprocessing_DATA.sav'
pickle.dump(tok, open(tok_file, 'wb'))

In [8]:
# for cnn preproces
max_len = 1000
cnn_texts_seq = tok.texts_to_sequences(texts)
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)


In [11]:
# try a simple model first

def get_simple_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(num_max,)))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc',metrics.binary_accuracy])
    print('compile done')
    return model

def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=2,verbose=1,validation_split=0.3)

m = get_simple_model()
check_model(m,mat_texts,tags)

#from sklearn.cross_validation import train_test_split
#X_train, X_test, Y_train, Y_test = train_test_split(cnn_texts_mat, tags, test_size=0.3)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               512512    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 644,097
Trainable params: 644,097
Non-trainable params: 0
_________________________________________________________________
compile done
Train on 2838 samples, validate on 1217 samples
Epoch 1/2
2838/2838 [==============================] - 3s 1ms/step - loss: 

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(cnn_texts_mat, tags, test_size=0.3)


# Evaluate model
scores = m.evaluate(X_test, Y_test, batch_size=128)
    
print("%s: %.2f%%" % (m.metrics_names[0], scores[0] * 100))
print("%s: %.2f%%" % (m.metrics_names[1], scores[1] * 100))
print("%s: %.2f%%" % (m.metrics_names[2], scores[1] * 100))

1217/1217 [==============================] - 0s 122us/step
loss: 753.99%
acc: 51.85%
binary_accuracy: 51.85%


In [13]:
m.save("CNN_model1_normal_embeddings_DATA_with_text_processing.sav")

In [14]:
text = np.array([' bad service'])
sequences = tok.texts_to_sequences(text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=num_max)
prediction = m.predict(sequences_matrix)
print(prediction)
if prediction >0.8 :
    print("NOTISSUE")
else :
    print("ISSUE")

[[1.]]
NOTISSUE


In [15]:
text = np.array([' mediocre service'])
sequences = tok.texts_to_sequences(text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=num_max)
prediction = m.predict(sequences_matrix)
print(prediction)
if prediction >0.8 :
    print("NOTISSUE")
else :
    print("ISSUE")

[[1.]]
NOTISSUE


In [16]:
text = np.array(['great service'])
sequences = tok.texts_to_sequences(text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=num_max)
prediction = m.predict(sequences_matrix)
print(prediction)
if prediction >0.8 :
    print("NOTISSUE")
else :
    print("ISSUE")

[[1.]]
NOTISSUE


In [17]:
# for cnn preproces
max_len = 1000
cnn_texts_seq = tok.texts_to_sequences(texts)
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)


In [18]:
def get_cnn_model_v1():   
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

m1 = get_cnn_model_v1()
check_model(m1,cnn_texts_mat,tags)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 20)          20000     
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 20)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 998, 64)           3904      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
__________

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(cnn_texts_mat, tags, test_size=0.3)


# Evaluate model
scores = m1.evaluate(X_test, Y_test, batch_size=128)
    
print("%s: %.2f%%" % (m1.metrics_names[0], scores[0] * 100))
print("%s: %.2f%%" % (m1.metrics_names[1], scores[1] * 100))
print("%s: %.2f%%" % (m1.metrics_names[2], scores[1] * 100))

1217/1217 [==============================] - 1s 607us/step
loss: 11.65%
acc: 96.71%
binary_accuracy: 96.71%


In [20]:
m1.save("CNN_model1_normal_embeddings_DATA_with_text_processing.sav")

In [21]:
text = np.array([' bad service'])
sequences = tok.texts_to_sequences(text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=num_max)
prediction = m1.predict(sequences_matrix)
print(prediction)
if prediction >0.8 :
    print("NOTISSUE")
else :
    print("ISSUE")

[[0.01772978]]
ISSUE


In [22]:
text = np.array(['good service'])
sequences = tok.texts_to_sequences(text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=num_max)
prediction = m1.predict(sequences_matrix)
print(prediction)
if prediction >0.8 :
    print("NOTISSUE")
else :
    print("ISSUE")


[[0.94345254]]
NOTISSUE


In [23]:
text = np.array(['help me please'])
sequences = tok.texts_to_sequences(text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=num_max)
prediction = m1.predict(sequences_matrix)
print(prediction)
if prediction >0.8 :
    print("NOTISSUE")
else :
    print("ISSUE")


[[0.28707814]]
ISSUE


In [24]:
def get_cnn_model_v2(): # added embed   
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        50, #!!!!!!!!!!!!!!!!!!!!!!!
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

m2 = get_cnn_model_v2()
check_model(m2,cnn_texts_mat,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 50)          50000     
_________________________________________________________________
dropout_7 (Dropout)          (None, 1000, 50)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 998, 64)           9664      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
__________

In [32]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(cnn_texts_mat, tags, test_size=0.3)


# Evaluate model
scores = m2.evaluate(X_test, Y_test, batch_size=128)
    
print("%s: %.2f%%" % (m2.metrics_names[0], scores[0] * 100))
print("%s: %.2f%%" % (m2.metrics_names[1], scores[1] * 100))
print("%s: %.2f%%" % (m2.metrics_names[2], scores[1] * 100))

1217/1217 [==============================] - 1s 1ms/step
loss: 5.74%
acc: 98.44%
binary_accuracy: 98.44%


In [34]:
m2.save("CNN_model2_normal_embeddings_DATA_with_text_processing.sav")

In [31]:
text = np.array(['help me'])
sequences = tok.texts_to_sequences(text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=num_max)
prediction = m2.predict(sequences_matrix)
print(prediction)
if prediction >0.8 :
    print("NOTISSUE")
else :
    print("ISSUE")


[[0.26738784]]
ISSUE


In [32]:
text = np.array([' bad service'])
sequences = tok.texts_to_sequences(text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=num_max)
prediction = m2.predict(sequences_matrix)
print(prediction)
if prediction >0.8 :
    print("NOTISSUE")
else :
    print("ISSUE")


[[0.03977815]]
ISSUE


In [33]:
def get_cnn_model_v3():    # added filter
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256, #!!!!!!!!!!!!!!!!!!!
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

m3 = get_cnn_model_v3()
check_model(m3,cnn_texts_mat,tags)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1000, 20)          20000     
_________________________________________________________________
dropout_11 (Dropout)         (None, 1000, 20)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 998, 256)          15616     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
__________

In [35]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(cnn_texts_mat, tags, test_size=0.3)


# Evaluate model
scores = m3.evaluate(X_test, Y_test, batch_size=128)
    
print("%s: %.2f%%" % (m3.metrics_names[0], scores[0] * 100))
print("%s: %.2f%%" % (m3.metrics_names[1], scores[1] * 100))
print("%s: %.2f%%" % (m3.metrics_names[2], scores[1] * 100))

1217/1217 [==============================] - 2s 2ms/step
loss: 35.37%
acc: 85.78%
binary_accuracy: 85.78%


In [36]:
m3.save("CNN_model3_normal_embeddings_DATA_with_text_processing.sav")

In [37]:
text = np.array(['help me'])
sequences = tok.texts_to_sequences(text)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=num_max)
prediction = m3.predict(sequences_matrix)
print(prediction)
if prediction >0.8 :
    print("NOTISSUE")
else :
    print("ISSUE")


[[0.39144567]]
ISSUE


In [38]:
# Save the model
m2.save('model2.h5')

# Recreate the exact same model purely from the file
#new_model = keras.models.load_model('path_to_my_model.h5')